<a href="https://colab.research.google.com/github/DilaraSaruhan/Natural-Language-Processing/blob/main/DilaraSaruhanNLPprojesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Doğal Dil İşleme Dersi - BIL570**

---


**Dilara Saruhan** ⭐

---


**DÖNEM PROJESİ**

**KONU:** Duygu Analizi İçin Sözlük Ve Model Oluşturma

▶ Linki aşağıda verilen sinema duygu analiz (pozitif, negatif ve nötür)  veri setini kullanarak 3 adet (pozitif, negatif ve nötür) her biri 50 kelimeden oluşan duygu sözlükleri oluşturmanız beklenmektedir (istediğiniz yöntemi kullanabilirsiniz).

Veri seti eğitim ve sınama için ikiye ayrılmıştır;

 1)Sözlüklerin eğitim veri seti kullanılarak oluşturulması,

 2)Elde edilen sözlükler kullanılarak sınama verilerinin duygularını maksimum tespit edecek bir model önerilmesi beklenmektedir.

https://ai.stanford.edu/~amaas/data/sentiment/

---

In [10]:
import os
import nltk
from nltk.corpus import stopwords
from string import punctuation
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

# Google Drive'ı '/content/drive' dizinine bağladım
drive.mount('/content/drive')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Dosya yolu
file_path = '/content/drive/MyDrive/Colab_Datasets/train/'

texts_and_labels = []  # Metinleri ve etiketlerini tutacak listeyi oluşturdum

# Dosyaları okuyup listede tutuyorum
for label in ["pos", "neg", "unsup"]:
    path = os.path.join(file_path, label)
    for file in os.listdir(path):
        file_full_path = os.path.join(path, file)
        try:
            with open(file_full_path, "r", encoding="utf-8") as f:
                text = f.read()
                texts_and_labels.append((text, label))
        except Exception as e:
            print(f"Error reading file {file_full_path}: {e}")

# Temizleme işleri için koşullarım
stop_words_and_punctuation = set(stopwords.words('english') + list(punctuation))
pos_words = []
neg_words = []
neu_words = []

for text, label in texts_and_labels:
    text = re.sub('<.*?>', '', text)
    words = nltk.word_tokenize(text)
    words = [w for w in words if not w in stop_words_and_punctuation]
    pos = nltk.pos_tag(words)
    words = [p[0].lower() for p in pos if p[1][0] == "J"]
    if label == "pos":
        pos_words.extend(words)
    elif label == "neg":
        neg_words.extend(words)
    else:
        neu_words.extend(words)

# Her bir kelime listesi için frekans dağılımı oluşturdum
pos_freq = nltk.FreqDist(pos_words)
neg_freq = nltk.FreqDist(neg_words)
neu_freq = nltk.FreqDist(neu_words)

# Her bir frekans dağılımından en sık geçen 50 kelimeyi seçtim
pos_dict = dict(pos_freq.most_common(50))
neg_dict = dict(neg_freq.most_common(50))
neu_dict = dict(neu_freq.most_common(50))

# Sözlükleri ekrana yazdırdım
print("Pozitif Duygu Sözlüğü:")
print(pos_dict)

print("\nNegatif Duygu Sözlüğü:")
print(neg_dict)

print("\nNötr Duygu Sözlüğü:")
print(neu_dict)


# Eğitim verilerini oluşturdum
X = [text for text, label in texts_and_labels]
y = [label for text, label in texts_and_labels]

# TF-IDF vektörleme işlemi yaptım
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

# Veriyi eğitim ve test setlerine böldüm
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Naive Bayes sınıflandırma modelini eğittim
model = MultinomialNB()
model.fit(X_train, y_train)

# Modelin performansını değerlendirdim
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

# Sınama verileri üzerinde modelin performansını gösterdim
test_file_path = '/content/drive/MyDrive/Colab_Datasets/test/'

test_texts_and_labels = []
for label in ["pos", "neg", "unsup"]:
    path = os.path.join(test_file_path, label)
    for file in os.listdir(path):
        with open(os.path.join(path, file), "r", encoding="utf-8") as f:
            text = f.read()
            test_texts_and_labels.append((text, label))

X_test_data = [text for text, label in test_texts_and_labels]
y_test_data = [label for text, label in test_texts_and_labels]

X_test_tfidf = vectorizer.transform(X_test_data)

# Modelin performansını değerlendirdim
y_test_pred = model.predict(X_test_tfidf)
accuracy_test = accuracy_score(y_test_data, y_test_pred)
classification_report_test = classification_report(y_test_data, y_test_pred)

print(f"\nModel Accuracy on Test Data: {accuracy_test}")
print(f"\nClassification Report on Test Data:\n{classification_report_test}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Pozitif Duygu Sözlüğü:
{'good': 7027, 'great': 5872, 'many': 3643, 'best': 3252, 'much': 3133, 'little': 2954, 'first': 2564, 'real': 2364, 'young': 2088, 'old': 1941, 'bad': 1753, 'new': 1674, 'last': 1501, 'big': 1469, 'different': 1464, 'funny': 1367, 'true': 1343, 'original': 1304, 'excellent': 1271, 'beautiful': 1255, 'whole': 1204, 'wonderful': 1137, 'main': 1009, 'classic': 1007, 'special': 947, 'nice': 945, 'small': 923, 'least': 917, 'sure': 912, 'hard': 907, 'right': 901, 'short': 880, 'better': 878, 'second': 859, 'human': 854, 'american': 848, 'high': 801, 'full': 789, 'long': 773, 'several': 761, 'perfect': 753, 'interesting': 740, 'early': 724, 'able': 713, 'strong': 702, 'final': 688, 'fine': 688, 'black': 679, 'favorite': 669, 'enough': 666}

Negatif Duygu Sözlüğü:
{'bad': 6762, 'good': 6708, 'much': 3308, 'many': 2826, 'little': 2757, 'great': 2469, 'first': 2208, 'real': 1995, 'old': 1902, 'original': 1867, 'worst': 1794, 'least': 1691, 'whole': 1656, 'best': 1583, 'f